For interacting with the new template format while drafting proposals.

In [1]:
#%%
# import os
# import re
# from datetime import datetime
# from pathlib import Path

import numpy as np
import pandas as pd
# import requests
import xlwings as xw

import functions
import hide
import checklist_collections as cc
import checklists

wb = xw.books.active
ws = wb.sheets.active
print(wb)   

<Book [TestTemplate.xlsx]>


In [ ]:
checklists.generate_proposal_checklist(wb)